In [1]:
import re
import pandas as pd
import bz2

### Download data

In [25]:
#C:/Users/User/Desktop/enwiki-latest-pages-articles10.xml-p4045403p5399366.bz2
file_name = "C:/Users/User/Desktop/enwiki-latest-pages-articles.xml.bz2"
#file_name = "C:/Users/User/Desktop/enwiki-latest-pages-articles10.xml-p4045403p5399366.bz2"

file = open("data/file_parse_whole_data.txt","w",encoding="utf8")
with bz2.open(file_name, "rb") as f:
    content = f.readline().decode('utf-8')
    while content:
        x = re.search("((={2,})( ?)(.*?)( ?)(={2,}))|<title>(.*)</title>", content)
        if x:
            y = re.search("\(\{\{[Nn]o [Mm]ore [Ll]inks\}\}\)",x.group())
            if y:
                content = f.readline().decode('utf-8')
                continue
            if x.group(3) or x.group(5):
                k = x.group(2) + x.group(4) + x.group(6) + "\n"
            else:
                k = x.group() + "\n"
            file.write(k)
        content = f.readline().decode('utf-8')
file.close()

"""
file = open("data/file_parse_whole_data.txt","w",encoding="utf8")
poc = 1
with bz2.open(file_name, "rb") as f:
    for line in f.readlines().decode('utf-8'):
        poc = poc + 1
        x = re.search("((={2,})( ?)(.*?)( ?)(={2,}))|<title>(.*)</title>", line)
        y = re.search("\(\{\{[Nn]o [Mm]ore [Ll]inks\}\}\)",x.group())
        if(y):
            continue
        if x.group(3) or x.group(5):
            k = x.group(2) + x.group(4) + x.group(6) + "\n"
        else:
            k = x.group() + "\n"
        file.write(k)
        if poc == 10:
            print(k)
        else:
            break
    
file.close()"""

'\nfile = open("data/file_parse_whole_data.txt","w",encoding="utf8")\npoc = 1\nwith bz2.open(file_name, "rb") as f:\n    for line in f.readlines().decode(\'utf-8\'):\n        poc = poc + 1\n        x = re.search("((={2,})( ?)(.*?)( ?)(={2,}))|<title>(.*)</title>", line)\n        y = re.search("\\(\\{\\{[Nn]o [Mm]ore [Ll]inks\\}\\}\\)",x.group())\n        if(y):\n            continue\n        if x.group(3) or x.group(5):\n            k = x.group(2) + x.group(4) + x.group(6) + "\n"\n        else:\n            k = x.group() + "\n"\n        file.write(k)\n        if poc == 10:\n            print(k)\n        else:\n            break\n    \nfile.close()'

In [26]:
def find_section_headers(text, file):
    for match in re.finditer("((={2,})( ?)(.*?)( ?)(={2,}))|<title>(.*)</title>",text):
        x = re.search("\(\{\{[Nn]o [Mm]ore [Ll]inks\}\}\)",match.group())
        if x:
            continue
        if match.group(3) or match.group(5):
            k = match.group(2) + match.group(4) + match.group(6) + "\n"
        else:
            k = match.group() + "\n"
        file.write(k)
    file.close()

In [2]:
def image_in_section(path):
    img = " ?\[\[File:[^\]]+\]\]"
    x = re.search(img,path)
    if x:
        path = re.sub(img,"",path)
    return path

In [3]:
def delete_linking(path):
    #x = re.search("\[\[(.*)?\|(.*)?\]\]|\[\[(.*)?\]\]",path)
    path = image_in_section(path)
    for x in re.finditer("\[\[([^\]]+)\|([^\]]+)\]\]|\[\[([^\]]+)\]\]",path):
        if x:
            if x.group(3):
                path = re.sub(re.escape(x.group()), re.escape(x.group(3)),path)
            elif x.group(2):
                path = re.sub(re.escape(x.group()), re.escape(x.group(2)),path)
   
    return path

In [4]:
def add(x, y):
    z = x + "\t" + y
    return z

In [5]:
def hierarchy(text, file_headers):
    for match in re.finditer("(={2,})(.*?)={2,}|<title>(.*)</title>",text):
        if match.group()[0] == "<":
            title = match.group(3)
            is_wiki_or_file = 0
            if (len(re.findall("Wikipedia:|File:|Help:|Template:",title)) > 0):
                is_wiki_or_file = 1
                continue
            level = 0
            path = title    
        elif (is_wiki_or_file == 0):
            if len(match.group(1)) > level:
                temp = path
                level = len(match.group(1))
                path = add(match.group(0),path)
                new_path = re.sub("=","",path);
                #new_path = image_in_section(new_path)
                new_path = delete_linking(new_path)
                file_headers.write(new_path + "\n")
                #print(path + "\n")

            elif len(match.group(1)) == level:
                path = add(match.group(0),temp)
                new_path = re.sub("=","",path);
                #new_path = image_in_section(new_path)
                new_path = delete_linking(new_path)
                file_headers.write(new_path + "\n")
                #print(path + "\n")

            else:
                while(1):
                    x = re.search("(={2,})(.*?)\t",temp)
                    if x:
                        if len(match.group(1)) <= len(x.group(1)):
                            temp = temp[(x.end()):]
                            if len(match.group(1)) == len(x.group(1)):
                                break
                        else:
                            break
                    else:
                        break

                level = len(match.group(1))
                path = add(match.group(0),temp)
                new_path = re.sub("=","",path);
                #new_path = image_in_section(new_path)
                new_path = delete_linking(new_path)
                file_headers.write(new_path + "\n")

                #print(path + "\n")

    file_headers.close()          

In [6]:
def edit_characters(file_unedit, text):
    x = re.sub("\'\'","",text)
    x = re.sub("&quot;","\"",x)
    x = re.sub("&amp;","&",x)
    x = re.sub("( ?\{\{[Aa]nchor ?\|[^\}]+\}\} ?|\{\{[Aa]nchor\}\})","",x)
    x = re.sub("&lt;span(?:.*?)&gt;&lt;/span&gt; ?","",x)
    #x = re.sub("&lt;ref(?:.*?)&lt;/ref&gt; ?","",x)
    #x = re.sub("&lt;ref(?:.*?)/&gt; ?","",x)
    x = re.sub("&lt;ref(?:.*?)(?:(/&gt;)|(&lt;/ref&gt;)) ?","",x)
    x = re.sub("&lt;\!--(?:.*?)--&gt; ?","",x)
    x = re.sub("&lt;(?:sub|sup)&gt;(.*?)&lt;/(?:sub|sup)&gt;",r'\1',x)
    x = re.sub("&lt;small&gt;(.*?)&lt;/small&gt;",r'\1',x)
    x = re.sub("&lt;span(?:.*?)&gt;(.*?)&lt;/span&gt;",r'\1',x)
    x = re.sub("{{rp\|[^}]*}}","",x)
    x = re.sub("&lt;u&gt;(.*?)&lt;/u&gt;",r'\1',x)
    x = re.sub("&lt;center&gt;(.*?)&lt;/center&gt;",r'\1',x)
    x = re.sub("&lt;big(?:.*?)&gt;(.*?)&lt;/big&gt;",r'\1',x)
    x = re.sub("&lt;math&gt;(.*?)&lt;/math&gt;",r'\1',x)
    x = re.sub(r"^\|(.*)","",x)
    file_unedit.write(x)
    file_unedit.close()

In [158]:
with bz2.open(file_name, "rb") as f:
    content = f.read().decode('utf-8')
    
file_parse_bigger_data = open("data/file_parse_bigger_data.txt","w",encoding="utf8")
find_section_headers(content, file_parse_bigger_data)

In [159]:
file = open("data/file_parse_bigger_data.txt", encoding="utf8")
text = file.read()
file.close()

file_headers = open("data/headers_bigger_data.txt","w",encoding="utf8")
hierarchy(text,file_headers)

In [167]:
file_unedited = open("data/headers_bigger_data.txt", encoding="utf8")
text = file_unedited.read()
file_unedited.close()

file_edit = open("data/headers_bigger_data_edited.txt","w",encoding="utf8")
edit_characters(file_edit, text)


In [3]:
from elasticsearch import Elasticsearch

In [4]:
es = Elasticsearch()

In [9]:
file = open("data/headers_bigger_data_edited.txt", encoding="utf8")
text = file.readlines()

In [18]:
poc = 1
for i in text:
    tmp = {"section":i}
    es.index(index="section_headers", doc_type="sections", id=poc, body=tmp)
    poc = poc + 1

C:\Users\User\Anaconda3\envs\iau\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [81]:
body = {
    "from":0,
    "track_total_hits": True,
    "query": {
        "match": {
            "section":"|titleArroyo"
        }
    }
}

res = es.search(index="section_headers", body=body)

In [25]:
len(text)

972532

In [82]:
for i in range(len(res['hits']['hits'])):
    print(res['hits']['hits'][i]['_source'])

{'section': '|titleArroyo Seco |publisherSecretaría de Turismo de Querétaro |locationQuerétaro |languageSpanish |accessdateApril 14, 2011 |archive-urlhttps://web.archive.org/web/20111003190422/http://www.queretaro.travel/fichatecnica.aspx?qKP71rlORWRV6+73qOfunuQ\tArroyo Seco, Querétaro\n'}


In [55]:
file_unedited = open("data/headers_bigger_data.txt", encoding="utf8")
text = file_unedited.read()
file_unedited.close()

In [76]:
file_edit = open("data/headers_bigger_data_edited_2.txt","w",encoding="utf8")
edit_characters(file_edit, text)

## Whole data

In [7]:
file = open("C:/Users/User/Desktop/file_parse_whole_data.txt", encoding="utf8")
text = file.read()
file.close()

file_headers = open("data/headers_whole_data.txt","w",encoding="utf8")
hierarchy(text,file_headers)

In [7]:
file_unedited = open("data/headers_whole_data.txt", encoding="utf8")
text = file_unedited.read()
file_unedited.close()

file_edit = open("data/headers_whole_data_edited.txt","w",encoding="utf8")
edit_characters(file_edit, text)